# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [ ]:
# If running from cloud-shell, DEVSHELL_PROJECT_ID environment variable may contain project ID.
# import os
# project_id = os.getenv("DEVSHELL_PROJECT_ID")

# Otherwise, set project_id manually
# project_id = 'XXXyourprojectID'

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
project_id = 'XXX-your-project-id'

# GSP 787
https://www.qwiklabs.com/focuses/11988?parent=catalog



### query 1

In [3]:
#%load_ext google.cloud.bigquery
#%%bigquery
from google.cloud import bigquery
client = bigquery.Client()
sql = """
SELECT
  source_year AS year,
  COUNT(is_male) AS birth_count
FROM `bigquery-public-data.samples.natality`
GROUP BY year
ORDER BY year DESC
LIMIT 15
"""
df = client.query(sql).to_dataframe()
df.head()

DefaultCredentialsError: ignored

### Task 1: Create a table partitioned by date

In [ ]:
from google.colab import syntax
import pandas as pd

query = syntax.sql('''
CREATE TABLE covid.oxford
PARTITION BY date
OPTIONS (partition_expiration_days=90)
AS SELECT * FROM `bigquery-public-data.covid19_govt_response.oxford_policy_tracker`
WHERE alpha_3_code NOT IN ('BGR', 'USA')
''')
pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,country_name,alpha_3_code,region_name,region_code,date,school_closing,school_closing_flag,school_closing_notes,workplace_closing,workplace_closing_flag,workplace_closing_notes,cancel_public_events,cancel_public_events_flag,cancel_public_events_notes,restrictions_on_gatherings,restrictions_on_gatherings_flag,restrictions_on_gatherings_notes,close_public_transit,close_public_transit_flag,close_public_transit_notes,stay_at_home_requirements,stay_at_home_requirements_flag,stay_at_home_requirements_notes,restrictions_on_internal_movement,restrictions_on_internal_movement_flag,restrictions_on_internal_movement_notes,international_travel_controls,international_travel_controls_notes,income_support,income_support_flag,income_support_notes,debt_contract_relief,debt_contract_relief_notes,fiscal_measures,fiscal_measures_notes,international_support,international_support_notes,public_information_campaigns,public_information_campaigns_flag,public_information_campaigns_notes,testing_policy,testing_policy_notes,contact_tracing,contact_tracing_notes,emergency_healthcare_investment,emergency_healthcare_investment_notes,vaccine_investment,vaccine_investment_notes,misc_wildcard,misc_wildcard_notes,confirmed_cases,deaths,stringency_index
0,Brazil,BRA,Distrito Federal,BR_DF,2020-11-10,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,NaN,NaN,NaN
1,Comoros,COM,None,None,2020-11-10,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,569.0,7.0,NaN
2,Afghanistan,AFG,None,None,2020-11-10,1.00,1,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,1.00,None,0.00,None,None,0.00,None,0.0,None,0.0,None,1.00,1,None,3.00,None,None,None,0.0,None,0.0,None,None,None,42463.0,1580.0,12.04
3,Burundi,BDI,None,None,2020-11-10,1.00,1,None,0.00,None,No policy changes found.,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,No policies found.,2.00,Quarantine measures stay in effect. Those who ...,1.00,0,None,0.00,None,0.0,None,0.0,None,1.00,1,None,1.00,None,1.00,None,0.0,None,0.0,None,None,None,615.0,1.0,14.81
4,Nicaragua,NIC,None,None,2020-11-10,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,1.00,None,0.00,None,None,0.00,None,0.0,None,0.0,None,1.00,1,None,1.00,None,0.00,None,0.0,None,0.0,None,None,None,5661.0,158.0,8.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20652,Sweden,SWE,None,None,2021-01-10,2.00,0,None,2.00,1,In addition to the measures mentioned in the p...,2.00,1,None,4.00,1,Whereas the 8-people limit was initially only ...,1.00,1,None,1.00,1,None,1.00,1,None,3.00,None,2.00,0,None,0.00,None,0.0,None,0.0,None,2.00,1,None,2.00,None,1.00,None,0.0,None,0.0,None,None,"On 10 January, a new temporary covid-19 law ca...",489471.0,9433.0,69.44
20653,Thailand,THA,None,None,2021-01-10,3.00,0,None,3.00,0,None,1.00,1,None,2.00,1,None,1.00,1,None,1.00,1,None,2.00,0,None,2.00,None,2.00,1,None,2.00,None,0.0,None,0.0,None,2.00,1,None,2.00,None,2.00,None,0.0,None,0.0,None,None,None,10547.0,67.0,63.89
20654,Turkey,TUR,None,None,2021-01-10,2.00,1,None,2.00,1,None,2.00,1,None,4.00,0,None,1.00,1,None,1.00,0,None,1.00,1,None,2.00,None,2.00,0,None,2.00,None,0.0,None,0.0,None,2.00,1,None,2.00,None,2.00,None,0.0,None,0.0,None,None,None,2326256.0,22807.0,65.28
20655,Uruguay,URY,None,None,2021-01-10,1.00,1,None,2.00,1,None,2.00,1,None,4.00,1,None,1.00,1,None,1.00,1,None,1.00,1,None,4.00,None,2.00,0,None,2.00,None,0.0,None,0.0,None,2.00,1,None,3.00,None,1.00,None,0.

### Task2: Add new columns to your table

In [ ]:


query = syntax.sql('''
ALTER TABLE covid.oxford
ADD COLUMN IF NOT EXISTS population INT64,
ADD COLUMN IF NOT EXISTS country_area FLOAT64,
ADD COLUMN IF NOT EXISTS mobility STRUCT<
avg_retail FLOAT64,
avg_grocery FLOAT64,
avg_parks FLOAT64,
avg_transit FLOAT64,
avg_workplace FLOAT64,
avg_residential FLOAT64
>
''')
pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,country_name,alpha_3_code,region_name,region_code,date,school_closing,school_closing_flag,school_closing_notes,workplace_closing,workplace_closing_flag,workplace_closing_notes,cancel_public_events,cancel_public_events_flag,cancel_public_events_notes,restrictions_on_gatherings,restrictions_on_gatherings_flag,restrictions_on_gatherings_notes,close_public_transit,close_public_transit_flag,close_public_transit_notes,stay_at_home_requirements,stay_at_home_requirements_flag,stay_at_home_requirements_notes,restrictions_on_internal_movement,restrictions_on_internal_movement_flag,restrictions_on_internal_movement_notes,international_travel_controls,international_travel_controls_notes,income_support,income_support_flag,income_support_notes,debt_contract_relief,debt_contract_relief_notes,fiscal_measures,fiscal_measures_notes,international_support,international_support_notes,public_information_campaigns,public_information_campaigns_flag,public_information_campaigns_notes,testing_policy,testing_policy_notes,contact_tracing,contact_tracing_notes,emergency_healthcare_investment,emergency_healthcare_investment_notes,vaccine_investment,vaccine_investment_notes,misc_wildcard,misc_wildcard_notes,confirmed_cases,deaths,stringency_index,population,country_area,mobility
0,Brazil,BRA,Distrito Federal,BR_DF,2020-11-10,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,NaN,NaN,NaN,None,NaN,None
1,Comoros,COM,None,None,2020-11-10,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,569.0,7.0,NaN,None,NaN,None
2,Afghanistan,AFG,None,None,2020-11-10,1.00,1,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,1.00,None,0.00,None,None,0.00,None,0.0,None,0.0,None,1.00,1,None,3.00,None,None,None,0.0,None,0.0,None,None,None,42463.0,1580.0,12.04,None,NaN,None
3,Burundi,BDI,None,None,2020-11-10,1.00,1,None,0.00,None,No policy changes found.,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,No policies found.,2.00,Quarantine measures stay in effect. Those who ...,1.00,0,None,0.00,None,0.0,None,0.0,None,1.00,1,None,1.00,None,1.00,None,0.0,None,0.0,None,None,None,615.0,1.0,14.81,None,NaN,None
4,Nicaragua,NIC,None,None,2020-11-10,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,0.00,None,None,1.00,None,0.00,None,None,0.00,None,0.0,None,0.0,None,1.00,1,None,1.00,None,0.00,None,0.0,None,0.0,None,None,None,5661.0,158.0,8.33,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20652,Sweden,SWE,None,None,2021-01-10,2.00,0,None,2.00,1,In addition to the measures mentioned in the p...,2.00,1,None,4.00,1,Whereas the 8-people limit was initially only ...,1.00,1,None,1.00,1,None,1.00,1,None,3.00,None,2.00,0,None,0.00,None,0.0,None,0.0,None,2.00,1,None,2.00,None,1.00,None,0.0,None,0.0,None,None,"On 10 January, a new temporary covid-19 law ca...",489471.0,9433.0,69.44,None,NaN,None
20653,Thailand,THA,None,None,2021-01-10,3.00,0,None,3.00,0,None,1.00,1,None,2.00,1,None,1.00,1,None,1.00,1,None,2.00,0,None,2.00,None,2.00,1,None,2.00,None,0.0,None,0.0,None,2.00,1,None,2.00,None,2.00,None,0.0,None,0.0,None,None,None,10547.0,67.0,63.89,None,NaN,None
20654,Turkey,TUR,None,None,2021-01-10,2.00,1,None,2.00,1,None,2.00,1,None,4.00,0,None,1.00,1,None,1.00,0,None,1.00,1,None,2.00,None,2.00,0,None,2.00,None,0.0,None,0.0,None,2.00,1,None,2.00,None,2.00,None,0.0,None,0.0,None,None,None,2326256.0,22807.0,65.28,None,NaN,None
20655,Uruguay,URY,None,None,2021-01-10,1.00,1,None

### Task3: Add country population data to the population column

In [ ]:
query = syntax.sql('''
UPDATE `covid.oxford` t0
SET t0.population = t1.pop_data_2019
FROM `bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide` t1
WHERE t0.date = t1.date AND t0.alpha_3_code=t1.country_territory_code
''')
pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,country_name,alpha_3_code,region_name,region_code,date,school_closing,school_closing_flag,school_closing_notes,workplace_closing,workplace_closing_flag,workplace_closing_notes,cancel_public_events,cancel_public_events_flag,cancel_public_events_notes,restrictions_on_gatherings,restrictions_on_gatherings_flag,restrictions_on_gatherings_notes,close_public_transit,close_public_transit_flag,close_public_transit_notes,stay_at_home_requirements,stay_at_home_requirements_flag,stay_at_home_requirements_notes,restrictions_on_internal_movement,restrictions_on_internal_movement_flag,restrictions_on_internal_movement_notes,international_travel_controls,international_travel_controls_notes,income_support,income_support_flag,income_support_notes,debt_contract_relief,debt_contract_relief_notes,fiscal_measures,fiscal_measures_notes,international_support,international_support_notes,public_information_campaigns,public_information_campaigns_flag,public_information_campaigns_notes,testing_policy,testing_policy_notes,contact_tracing,contact_tracing_notes,emergency_healthcare_investment,emergency_healthcare_investment_notes,vaccine_investment,vaccine_investment_notes,misc_wildcard,misc_wildcard_notes,confirmed_cases,deaths,stringency_index,population,country_area,mobility
0,Brazil,BRA,Alagoas,BR_AL,2020-12-31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,104818.0,2489.0,NaN,NaN,NaN,None
1,Brazil,BRA,Distrito Federal,BR_DF,2020-12-31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,251701.0,4259.0,NaN,NaN,NaN,None
2,Brazil,BRA,Rio de Janeiro,BR_RJ,2020-12-31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,434648.0,25530.0,NaN,NaN,NaN,None
3,Brazil,BRA,Santa Catarina,BR_SC,2020-12-31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,492583.0,5253.0,NaN,NaN,NaN,None
4,Comoros,COM,None,None,2020-12-31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,823.0,10.0,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20652,Kosovo,RKS,None,None,2020-12-07,1.00,1,No change found.,2.00,1,No change found.,2.00,1,No change found.,4.00,1,"No change. ""All public gatherings are proh...",1.00,1,No change found.,2.00,1,No change found.,2.00,1,No change found.,1.00,None,1.00,0,None,2.00,No change found.,0.0,No measures found.,0.0,No measures found.,2.00,1,No change found.,3.00,No change found.,2.00,No change found.,0.0,No measures found.,0.0,No measures found.,None,None,43324.0,1120.0,71.30,NaN,NaN,None
20653,Malaysia,MYS,None,None,2020-12-07,3.00,1,Malaysia continues to adjust restrictions to s...,2.00,1,"For the more relaxed RMCO states, Most busines...",1.00,1,"Under all levels of all CMCO, Authorities cont...",4.00,0,"For EMCO states, such as Kuala Lumpur and Saba...",1.00,1,Public transport can operate with protocols in...,2.00,0,"For EMCO states, such as Kuala Lumpur and Saba...",1.00,0,Travellers no longer need to obtain police per...,3.00,Travel Restrictions most foreigners remain ban...,1.00,1,None,2.00,None,0.0,None,0.0,Non

### Task4: Add country area data to the country_area column

In [ ]:
query = syntax.sql('''
UPDATE `covid.oxford` t0
SET t0.country_area = t1.country_area
FROM `bigquery-public-data.census_bureau_international.country_names_area` t1
WHERE t0.country_name = t1.country_name
''')
pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,country_name,alpha_3_code,region_name,region_code,date,school_closing,school_closing_flag,school_closing_notes,workplace_closing,workplace_closing_flag,workplace_closing_notes,cancel_public_events,cancel_public_events_flag,cancel_public_events_notes,restrictions_on_gatherings,restrictions_on_gatherings_flag,restrictions_on_gatherings_notes,close_public_transit,close_public_transit_flag,close_public_transit_notes,stay_at_home_requirements,stay_at_home_requirements_flag,stay_at_home_requirements_notes,restrictions_on_internal_movement,restrictions_on_internal_movement_flag,restrictions_on_internal_movement_notes,international_travel_controls,international_travel_controls_notes,income_support,income_support_flag,income_support_notes,debt_contract_relief,debt_contract_relief_notes,fiscal_measures,fiscal_measures_notes,international_support,international_support_notes,public_information_campaigns,public_information_campaigns_flag,public_information_campaigns_notes,testing_policy,testing_policy_notes,contact_tracing,contact_tracing_notes,emergency_healthcare_investment,emergency_healthcare_investment_notes,vaccine_investment,vaccine_investment_notes,misc_wildcard,misc_wildcard_notes,confirmed_cases,deaths,stringency_index,population,country_area,mobility
0,Brazil,BRA,Alagoas,BR_AL,2020-12-28,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,103748.0,2468.0,NaN,NaN,8358140.0,None
1,Brazil,BRA,Distrito Federal,BR_DF,2020-12-28,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,249736.0,4223.0,NaN,NaN,8358140.0,None
2,Brazil,BRA,Rio de Janeiro,BR_RJ,2020-12-28,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,421674.0,24942.0,NaN,NaN,8358140.0,None
3,Brazil,BRA,Santa Catarina,BR_SC,2020-12-28,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,482129.0,5082.0,NaN,NaN,8358140.0,None
4,Comoros,COM,None,None,2020-12-28,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,715.0,7.0,NaN,NaN,2235.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20652,Spain,ESP,None,None,2020-12-26,3.00,1,None,2.00,1,None,2.00,1,None,4.00,1,None,0.00,None,None,2.00,1,None,2.00,1,None,3.00,None,2.00,0,None,2.00,None,0.0,None,0.0,None,2.00,1,None,2.00,None,1.00,None,0.0,None,0.0,None,None,None,1854951.0,49824.0,78.70,NaN,498980.0,None
20653,France,FRA,None,None,2020-12-26,2.00,1,None,2.00,1,None,2.00,1,None,4.00,1,None,0.00,None,None,2.00,1,None,0.00,None,None,3.00,None,2.00,1,None,0.00,None,0.0,None,0.0,None,2.00,1,None,3.00,None,2.00,None,0.0,None,0.0,None,None,None,2548388.0,62209.0,63.89,NaN,640427.0,None
20654,United Kingdom,GBR,Wales,UK_WAL,2020-12-26,2.00,1,None,3.00,1,Wales Stay at Home restrictions back in place ...,2.00,1,None,4.00,1,Wales Stay at Home restrictions back in place ...,1.00,1,None,2.00,1,Wales Stay at Home restrictions back in place ...,2.00,1,Wales Stay at Home restrictions back in place ...,3.00,None,2.00,1,None,2.00,None,0.0,None,0.0,None,2.00,1,None,2.00,None,1.00,None,0.0,None,0.0,None,None,None,143379.0,3298.0,84.26

### Task5: Populate the mobility record data

In [ ]:
query = syntax.sql('''
UPDATE `covid.oxford` t0
SET t0.mobility = STRUCT<
avg_retail FLOAT64, avg_grocery FLOAT64, avg_parks FLOAT64, avg_transit FLOAT64,
avg_workplace FLOAT64, avg_residential FLOAT64
>
(t1.avg_retail, t1.avg_grocery, t1.avg_parks, t1.avg_transit,
t1.avg_workplace, t1.avg_residential)
FROM ( SELECT country_region, date,
  AVG(retail_and_recreation_percent_change_from_baseline) as avg_retail,
  AVG(grocery_and_pharmacy_percent_change_from_baseline) as avg_grocery,
  AVG(parks_percent_change_from_baseline) as avg_parks,
  AVG(transit_stations_percent_change_from_baseline) as avg_transit,
  AVG(workplaces_percent_change_from_baseline) as avg_workplace,
  AVG(residential_percent_change_from_baseline) as avg_residential
  FROM `bigquery-public-data.covid19_google_mobility.mobility_report`
  GROUP BY country_region, date) AS t1
WHERE t0.date = t1.date AND t0.country_name=t1.country_region
''')
pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,country_name,alpha_3_code,region_name,region_code,date,school_closing,school_closing_flag,school_closing_notes,workplace_closing,workplace_closing_flag,workplace_closing_notes,cancel_public_events,cancel_public_events_flag,cancel_public_events_notes,restrictions_on_gatherings,restrictions_on_gatherings_flag,restrictions_on_gatherings_notes,close_public_transit,close_public_transit_flag,close_public_transit_notes,stay_at_home_requirements,stay_at_home_requirements_flag,stay_at_home_requirements_notes,restrictions_on_internal_movement,restrictions_on_internal_movement_flag,restrictions_on_internal_movement_notes,international_travel_controls,international_travel_controls_notes,income_support,income_support_flag,income_support_notes,debt_contract_relief,debt_contract_relief_notes,fiscal_measures,fiscal_measures_notes,international_support,international_support_notes,public_information_campaigns,public_information_campaigns_flag,public_information_campaigns_notes,testing_policy,testing_policy_notes,contact_tracing,contact_tracing_notes,emergency_healthcare_investment,emergency_healthcare_investment_notes,vaccine_investment,vaccine_investment_notes,misc_wildcard,misc_wildcard_notes,confirmed_cases,deaths,stringency_index,population,country_area,mobility
0,Aruba,ABW,None,None,2021-02-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,6986.0,59.0,40.74,NaN,180.0,None
1,Albania,ALB,None,None,2021-02-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,78992.0,1393.0,60.19,NaN,27398.0,None
2,Andorra,AND,None,None,2021-02-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,9972.0,101.0,52.78,NaN,468.0,None
3,United Arab Emirates,ARE,None,None,2021-02-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,306339.0,859.0,50.93,NaN,83600.0,None
4,Argentina,ARG,None,None,2021-02-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,1933853.0,48249.0,79.17,NaN,2736690.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20652,Dominica,DMA,None,None,2020-12-07,1.00,1,The schools are fully open with social distanc...,1.00,1,"Establishments such as restaurants and bars, w...",2.00,1,"“Regarding the gathering of people, it has bee...",3.00,1,According to the Director of Communication and...,0.00,None,Masks are required in public transportation ac...,2.00,1,At the press conference held in the Green Room...,0.00,None,There are no known restrictions to movement. P...,2.00,According to the UK government's website on tr...,1.00,1,There is the Livelihood Grant Assistance Schem...,1.00,There was a price of petroleum products releas...,0.0,There was no new information gathered for the ...,0.0,There was no new information gathered for the ...,2.00,1,There is a public campaign and government offi...,2.00,According to the US Embassy in Dominica's webs...,2.00,There seems to be a rigorous contact tracing m...,0.0,There is no new information for the timeframe.,0.0,There is no new information for the timeframe.,None,None,85.0,0.0,50.93,71808

### Task6: Query missing data in population and country_area columns

In [ ]:
query = syntax.sql('''
SELECT DISTINCT country_name
FROM `covid.oxford`
WHERE population IS NULL
UNION ALL
SELECT DISTINCT country_name
FROM `covid.oxford`
WHERE country_area IS NULL
ORDER BY country_name ASC
''')
pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,country_name
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
192,Venezuela
193,Vietnam
194,Yemen
195,Zambia
